### DTS - Complete Streets

# 0x - Economic Justice/Sea Level Rise

**Author:** rmangan

**Purpose:**

Implement Modal Priortizaiton logic for street segments that have Unconstrained Modal Width > ROW Width estimate. 

**This script performs the following functions:**

1. Pre-process data - add all necessary fields, copy unconstrained values to new constrained fields

2. Process Boulevards

3. Process Avenues

4. Process Main Streets

5. Process Major Streets

6. Process Streets/Lanes/Alleys

7. Cleanup (if needed)

8. Results Analysis

9. Join Data to Feature Services (if needed)

3. (optional utility functions) Resent all typologies to NULL for reprocessing


**Global Assumptions and Notes:**
1. All typology comments with values in "Note" field are skipped, comments must be manually addressed in ArcGIS Pro.
2. item
3. item
4. item

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - used for common data geo-processing
2. arcgis 1.83 - used for connecting to AGOL org and related items
2. pandas 1.1 - (optional) used for QC purposess and data exploration

In [ ]:
# import modules
import arcpy
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS


In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

EJ = r"Z:\H\Honolulu_DTS\D3409300_RailActivation\Transfer\Communications\From\External\20211005_DTS_T6EJ\T6EJ_Data.gdb\OahuMPO_EJ_Areas_Updated_2016_Block_Groups"

SLR = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb\SLR"

In [ ]:
#Create DF analysis
ej_df = pd.DataFrame.spatial.from_featureclass(EJ)

df_fields = df.columns.values.tolist()

df.columns


In [ ]:
#Create DF analysis
df = pd.DataFrame.spatial.from_featureclass(modal_composite)

df_fields = ["SEGMENTID","modal_area_bike_exist","modal_area_bike_prop","modal_area_bike_const", "length_ft"]

f_bike_e = df["modal_area_bike_exist"].notnull()
f_bike_p = df["modal_area_bike_prop"].notnull()
f_bike_c = df["modal_area_bike_const"].notnull()

f_bikew_e = df["modal_width_bike_exist"].notnull()
f_bikew_p = df["modal_width_bike_prop"].notnull()
f_bikew_c = df["modal_width_bike_const"].notnull()

f_owner = (df["OWNER"] == "CITY") | (df["OWNER"] == "City") | (df["OWNER"] == "VARIOUS")

# df.loc[f_owner & f_bike,df_fields].shape
bike_area_e = round(df.loc[f_owner & f_bike_e,df_fields].sum()[1]/43560)
bike_area_p = round(df.loc[f_owner & f_bike_p,df_fields].sum()[2]/43560)
bike_area_c = round(df.loc[f_owner & f_bike_c,df_fields].sum()[3]/43560)

bike_length_e = round(df.loc[f_owner & f_bikew_e,df_fields].sum()[4]/5280)
bike_length_p = round(df.loc[f_owner & f_bikew_p,df_fields].sum()[4]/5280)
bike_length_c = round(df.loc[f_owner & f_bikew_c,df_fields].sum()[4]/5280)


bike_records_e = df.loc[f_owner & f_bike_e,df_fields].shape[0]
bike_records_p = df.loc[f_owner & f_bike_p,df_fields].shape[0]
bike_records_c = df.loc[f_owner & f_bike_c,df_fields].shape[0]

print("Bike Existing Records: {}".format(bike_records_e))
print("Bike Existing Miles: {}".format(bike_length_e))
print("Bike Existing Area: {}".format(bike_area_e))
print("")
print("Bike Proposed Records: {}".format(bike_records_p))
print("Bike Proposed Miles: {}".format(bike_length_p))
print("Bike Proposed Area: {}".format(bike_area_p))
print("")
print("Bike Constrained Records: {}".format(bike_records_c))
print("Bike Constrained Miles: {}".format(bike_length_c))
print("Bike Constrained Area: {}".format(bike_area_c))

df_bike_p = df.loc[f_owner & f_bike_p,["modal_area_bike_exist","modal_area_bike_prop", "modal_area_bike_const"]]

df_bike_p.plot.hist(subplots=True, sharey="col", bins = 20,figsize = (20,10))

In [ ]:
#Pandas DF operations


#rename land value field
df = df.rename(columns={"LandValue":"Land Value Metric"})

#calc row future area
df["row_area_future"] = df["row_width_future"] * df["length_ft"]


#add df fields for land value estimates
#land value metric is median land value ($/sq.ft) of all parcels within 1/4 mile of street segment

#total ROW
df["row_LV"] = df["mean_row_area"] * df["Land Value Metric"]
#future ROW (if 1986 future greater than 2020 estimate)
df["row_future_LV"] = df["row_area_future"] * df["Land Value Metric"]


#auto
df["modal_area_auto_exist_LV"] = df["modal_area_auto_exist"] * df["Land Value Metric"]
df["modal_area_auto_prop_LV"] = df["modal_area_auto_prop"] * df["Land Value Metric"]
df["modal_area_auto_const_LV"] = df["modal_area_auto_const"] * df["Land Value Metric"]


#bike
df["modal_area_bike_exist_LV"] = df["modal_area_bike_exist"] * df["Land Value Metric"]
df["modal_area_bike_prop_LV"] = df["modal_area_bike_prop"] * df["Land Value Metric"]
df["modal_area_bike_const_LV"] = df["modal_area_bike_const"] * df["Land Value Metric"]


#medians
df["modal_area_medians_exist_LV"] = df["modal_area_medians_exist"] * df["Land Value Metric"]
df["modal_area_medians_prop_LV"] = df["modal_area_medians_prop"] * df["Land Value Metric"]
df["modal_area_medians_const_LV"] = df["modal_area_medians_const"] * df["Land Value Metric"]


#parking
df["modal_area_park_exist_LV"] = df["modal_area_park_exist"] * df["Land Value Metric"]
df["modal_area_park_prop_LV"] = df["modal_area_park_prop"] * df["Land Value Metric"]
df["modal_area_park_const_LV"] = df["modal_area_park_const"] * df["Land Value Metric"]


#Ped
df["modal_area_ped_exist_LV"] = df["modal_area_ped_exist"] * df["Land Value Metric"]
df["modal_area_ped_prop_LV"] = df["modal_area_ped_prop"] * df["Land Value Metric"]
df["modal_area_ped_const_LV"] = df["modal_area_ped_const"] * df["Land Value Metric"]


#Bus
df["modal_area_bus_exist_LV"] = df["modal_area_bus_exist"] * df["Land Value Metric"]
df["modal_area_bus_prop_LV"] = df["modal_area_bus_prop"] * df["Land Value Metric"]
df["modal_area_bus_const_LV"] = df["modal_area_bus_const"] * df["Land Value Metric"]

In [ ]:
def percent(part,total):
    #return percent
    percent = round(((part/total)*100),3)
    return percent


percent(2521,10330)

In [ ]:
fields = [
    "SEGMENTID",              #0
    "length_ft",              #1
    "FULLNAME",               #2
    "CS_type",                #3      
    "mean_row_2020",          #4
    "modal_width_max",        #5
    "modal_width_max_diff",   #6
    "modal_width_const",      #7
    "modal_width_const_diff", #8          
    "modal_area_auto_exist",  #9
    "modal_area_auto_prop",   #10
    "modal_area_auto_const",  #11         
    "modal_area_bike_exist",  #12
    "modal_area_bike_prop",   #13
    "modal_area_bike_const",  #14         
    "modal_area_park_exist",  #15
    "modal_area_park_prop",   #16
    "modal_area_park_const",  #17         
    "modal_area_ped_exist",   #18
    "modal_area_ped_prop",    #19
    "modal_area_ped_const",   #20
    "mean_row_area",           #21
    "modal_area_medians_exist",  #22
    "modal_area_medians_prop",   #23
    "modal_area_medians_const",  #24         
    "modal_area_bus_exist",  #25
    "modal_area_bus_prop",   #26
    "modal_area_bus_const",  #27
    "row_area_future"   #28
]


#Filters for Dataframe
f_owner = (df["OWNER"] == "CITY") | (df["OWNER"] == "City") | (df["OWNER"] == "VARIOUS")




f_bike_exist_sharedroad = (df["Fac_Type_BE"] == "Shared Roadway")
f_bike_exist_sharedpath = (df["Fac_Type_BE"] == "Shared Use Path")

f_bike_exist_distinct = ~ ((df["Fac_Type_BE"] == "Shared Roadway") | (df["Fac_Type_BE"] == "Shared Use Path"))

f_bike_prop_sharedroad = (df["Fac_Type_BP"] == "Shared Roadway")
f_bike_redev_sharedroad = (df["Fac_Type_BR"] == "Shared Roadway")

f_bike_prop_sharedpath = (df["Fac_Type_BP"] == "Shared Use Path")
f_bike_redev_sharedpath = (df["Fac_Type_BR"] == "Shared Use Path")

f_bike_prop_distinct = ~(f_bike_prop_sharedroad  | f_bike_redev_sharedroad | f_bike_prop_sharedpath | f_bike_redev_sharedpath )



f_reductions = (df["modal_width_max"] > df["modal_width_const"])

f_unconst_over = df["modal_width_max_diff"] >= 0
f_unconst_under = df["modal_width_max_diff"] <= 0

f_const_over = df["modal_width_const_diff"] >= 0
f_const_under = df["modal_width_const_diff"] <= 0

f_ej = df["ej_percent"] > .5
f_slr = df["SLR_percent"] > .5


def percent(part,total):
    #return percent
    percent = round(((part/total)*100),3)
    return percent


#Total Metrics
total_miles = round(df.loc[:, fields].sum()[1]/5280)
total_miles_ej = round(df.loc[f_ej, fields].sum()[1]/5280)
total_miles_slr = round(df.loc[f_slr, fields].sum()[1]/5280)
total_area = round(df.loc[:, fields].sum()[21]/43560)

#Project (Owner = City & Various)
project_miles = round(df.loc[f_owner, fields].sum()[1]/5280)
project_miles_ej = round(df.loc[f_owner & f_ej, fields].sum()[1]/5280)
project_miles_slr = round(df.loc[f_owner & f_slr, fields].sum()[1]/5280)
project_area = round(df.loc[f_owner, fields].sum()[21]/43560)
project_area_future = round(df.loc[f_owner, fields].sum()[28]/43560)


unconst_over = round(df.loc[f_unconst_over & f_owner,fields].sum()[1]/5280)
unconst_under = round(df.loc[f_unconst_under & f_owner,fields].sum()[1]/5280)

const_over = round(df.loc[f_const_over & f_owner,fields].sum()[1]/5280)
const_under = round(df.loc[f_const_under & f_owner,fields].sum()[1]/5280)

land_value = (round(df["row_LV"].sum()/1000000))
land_value_future = (round(df["row_future_LV"].sum()/1000000))

land_value_proj = (round(df.loc[f_owner,"row_LV"].sum()/1000000))
land_value_proj_future = (round(df.loc[f_owner,"row_future_LV"].sum()/1000000))



#Modal Metrics
auto_area_exist = round(df.loc[f_owner,fields].sum()[9]/43560)
auto_area_exist_ej = round(df.loc[f_owner & f_ej,fields].sum()[9]/43560)
auto_area_exist_slr = round(df.loc[f_owner & f_slr,fields].sum()[9]/43560)

auto_area_prop = round(df.loc[f_owner,fields].sum()[10]/43560)
auto_area_prop_ej = round(df.loc[f_owner & f_ej,fields].sum()[10]/43560)
auto_area_prop_slr = round(df.loc[f_owner & f_slr,fields].sum()[10]/43560)

auto_area_const = round(df.loc[f_owner,fields].sum()[11]/43560)
auto_area_const_ej = round(df.loc[f_owner & f_ej,fields].sum()[11]/43560)
auto_area_const_slr = round(df.loc[f_owner & f_slr,fields].sum()[11]/43560)

auto_current_value = (round(df.loc[f_owner,"modal_area_auto_exist_LV"].sum()/1000000))
auto_prop_value = (round(df.loc[f_owner,"modal_area_auto_prop_LV"].sum()/1000000))
auto_const_value = (round(df.loc[f_owner,"modal_area_auto_const_LV"].sum()/1000000))



bike_area_exist = round(df.loc[f_owner,fields].sum()[12]/43560)
bike_area_exist_distinct = round(df.loc[f_owner & f_bike_exist_distinct,fields].sum()[12]/43560)


bike_area_exist_sharedroad = round(df.loc[f_owner & f_bike_exist_sharedroad,fields].sum()[12]/43560)
bike_area_exist_sharedpath = round(df.loc[f_owner & f_bike_exist_sharedpath,fields].sum()[12]/43560)

bike_area_exist_ej = round(df.loc[f_owner & f_ej,fields].sum()[12]/43560)
bike_area_exist_slr = round(df.loc[f_owner & f_slr,fields].sum()[12]/43560)

bike_area_exist_distinct_ej = round(df.loc[f_owner & f_bike_exist_distinct & f_ej,fields].sum()[12]/43560)
bike_area_exist_distinct_slr = round(df.loc[f_owner & f_bike_exist_distinct & f_slr,fields].sum()[12]/43560)



bike_area_prop = round(df.loc[f_owner,fields].sum()[13]/43560)
bike_area_prop_distinct = round(df.loc[f_owner & f_bike_prop_distinct,fields].sum()[13]/43560)

bike_area_prop_sharedroad = round(df.loc[f_owner & (f_bike_prop_sharedroad | f_bike_redev_sharedroad),fields].sum()[13]/43560)
bike_area_prop_sharedpath = round(df.loc[f_owner & (f_bike_prop_sharedpath | f_bike_redev_sharedpath),fields].sum()[13]/43560)

bike_area_prop_ej = round(df.loc[f_owner & f_ej,fields].sum()[13]/43560)
bike_area_prop_distinct_ej = round(df.loc[f_owner & f_bike_prop_distinct & f_ej,fields].sum()[13]/43560)
bike_area_prop_slr = round(df.loc[f_owner & f_slr,fields].sum()[13]/43560)
bike_area_prop_distinct_slr = round(df.loc[f_owner & f_bike_prop_distinct & f_slr,fields].sum()[13]/43560)




bike_area_const = round(df.loc[f_owner,fields].sum()[14]/43560)
bike_area_const_distinct = round(df.loc[f_owner & f_bike_prop_distinct,fields].sum()[14]/43560)

bike_area_const_sharedroad = round(df.loc[f_owner & (f_bike_prop_sharedroad | f_bike_redev_sharedroad),fields].sum()[14]/43560)
bike_area_const_sharedpath = round(df.loc[f_owner & (f_bike_prop_sharedpath | f_bike_redev_sharedpath),fields].sum()[14]/43560)

bike_area_const_ej = round(df.loc[f_owner & f_ej,fields].sum()[14]/43560)
bike_area_const_distinct_ej = round(df.loc[f_owner & f_bike_prop_distinct & f_ej,fields].sum()[14]/43560)
bike_area_const_slr = round(df.loc[f_owner & f_slr,fields].sum()[14]/43560)
bike_area_const_distinct_slr = round(df.loc[f_owner & f_bike_prop_distinct & f_slr,fields].sum()[14]/4356)
                            
bike_current_value = (round(df.loc[f_owner,"modal_area_bike_exist_LV"].sum()/1000000))
bike_distinct_current_value = (round(df.loc[f_owner & f_bike_exist_distinct,"modal_area_bike_exist_LV"].sum()/1000000))

bike_prop_value = (round(df.loc[f_owner,"modal_area_bike_prop_LV"].sum()/1000000))
bike_prop_distinct_value = (round(df.loc[f_owner & f_bike_prop_distinct,"modal_area_bike_prop_LV"].sum()/1000000))

bike_const_value = (round(df.loc[f_owner,"modal_area_bike_const_LV"].sum()/1000000))
bike_const_distinct_value = (round(df.loc[f_owner & f_bike_prop_distinct,"modal_area_bike_const_LV"].sum()/1000000))


medians_area_exist = round(df.loc[f_owner,fields].sum()[22]/43560)
medians_area_exist_ej = round(df.loc[f_owner & f_ej,fields].sum()[22]/43560)
medians_area_exist_slr = round(df.loc[f_owner & f_slr,fields].sum()[22]/43560)

medians_area_prop = round(df.loc[f_owner,fields].sum()[23]/43560)
medians_area_prop_ej = round(df.loc[f_owner & f_ej,fields].sum()[23]/43560)
medians_area_prop_slr = round(df.loc[f_owner & f_slr,fields].sum()[23]/43560)

medians_area_const = round(df.loc[f_owner,fields].sum()[24]/43560)
medians_area_const_ej = round(df.loc[f_owner & f_ej,fields].sum()[24]/43560)
medians_area_const_slr = round(df.loc[f_owner & f_slr,fields].sum()[24]/43560)

medians_current_value = (round(df.loc[f_owner,"modal_area_medians_exist_LV"].sum()/1000000))
medians_prop_value = (round(df.loc[f_owner,"modal_area_medians_prop_LV"].sum()/1000000))
medians_const_value = (round(df.loc[f_owner,"modal_area_medians_const_LV"].sum()/1000000))



park_area_exist = round(df.loc[f_owner,fields].sum()[15]/43560)
park_area_exist_ej = round(df.loc[f_owner & f_ej,fields].sum()[15]/43560)
park_area_exist_slr = round(df.loc[f_owner & f_slr,fields].sum()[15]/43560)

park_area_prop = round(df.loc[f_owner,fields].sum()[16]/43560)
park_area_prop_ej = round(df.loc[f_owner & f_ej,fields].sum()[16]/43560)
park_area_prop_slr = round(df.loc[f_owner & f_slr,fields].sum()[16]/43560)

park_area_const = round(df.loc[f_owner,fields].sum()[17]/43560)
park_area_const_ej = round(df.loc[f_owner & f_ej,fields].sum()[17]/43560)
park_area_const_slr = round(df.loc[f_owner & f_slr,fields].sum()[17]/43560)

park_current_value = (round(df.loc[f_owner,"modal_area_park_exist_LV"].sum()/1000000))
park_prop_value = (round(df.loc[f_owner,"modal_area_park_prop_LV"].sum()/1000000))
park_const_value = (round(df.loc[f_owner,"modal_area_park_const_LV"].sum()/1000000))



ped_area_exist = round(df.loc[f_owner,fields].sum()[18]/43560)
ped_area_exist_ej = round(df.loc[f_owner & f_ej,fields].sum()[18]/43560)
ped_area_exist_slr = round(df.loc[f_owner & f_slr,fields].sum()[18]/43560)

ped_area_prop = round(df.loc[f_owner,fields].sum()[19]/43560)
ped_area_prop_ej = round(df.loc[f_owner & f_ej,fields].sum()[19]/43560)
ped_area_prop_slr = round(df.loc[f_owner & f_slr,fields].sum()[19]/43560)

ped_area_const = round(df.loc[f_owner,fields].sum()[20]/43560)
ped_area_const_ej = round(df.loc[f_owner & f_ej,fields].sum()[20]/43560)
ped_area_const_slr = round(df.loc[f_owner & f_slr,fields].sum()[20]/43560)

ped_current_value = (round(df.loc[f_owner,"modal_area_ped_exist_LV"].sum()/1000000))
ped_prop_value = (round(df.loc[f_owner,"modal_area_ped_prop_LV"].sum()/1000000))
ped_const_value = (round(df.loc[f_owner,"modal_area_ped_const_LV"].sum()/1000000))



bus_area_exist = round(df.loc[f_owner,fields].sum()[25]/43560)
bus_area_exist_ej = round(df.loc[f_owner & f_ej,fields].sum()[25]/43560)
bus_area_exist_slr = round(df.loc[f_owner & f_slr,fields].sum()[25]/43560)

bus_area_prop = round(df.loc[f_owner,fields].sum()[26]/43560)
bus_area_prop_ej = round(df.loc[f_owner & f_ej,fields].sum()[26]/43560)
bus_area_prop_slr = round(df.loc[f_owner & f_slr,fields].sum()[26]/43560)

bus_area_const = round(df.loc[f_owner,fields].sum()[27]/43560)
bus_area_const_ej = round(df.loc[f_owner & f_ej,fields].sum()[27]/43560)
bus_area_const_slr = round(df.loc[f_owner & f_slr,fields].sum()[27]/43560)

bus_current_value = (round(df.loc[f_owner,"modal_area_bus_exist_LV"].sum()/1000000))
bus_prop_value = (round(df.loc[f_owner,"modal_area_bus_prop_LV"].sum()/1000000))
bus_const_value = (round(df.loc[f_owner,"modal_area_bus_const_LV"].sum()/1000000))


print("\n#Totals (all data)#")
print("Miles: {0:,}".format(total_miles))
print("Acres: {0:,} (${1:,}M)".format(total_area,land_value))
print("EJ: {0:,} ({1}%)".format(total_miles_ej, percent(total_miles_ej, total_miles)))
print("SLR: {0:,} ({1}%)".format(total_miles_slr,percent(total_miles_slr,total_miles)))


print("\n#Totals (City & Various)#")
print("Miles: {0:,}".format(project_miles))
print("Acres: {0:,} (${1:,}M)".format(project_area,land_value_proj))
print("Acres (future): {0:,} (${1:,}M)".format(project_area_future,land_value_proj))
print("EJ: {0:,} ({1}%)".format(project_miles_ej, percent(project_miles_ej, project_miles)))
print("SLR: {0:,} ({1}%)".format(project_miles_slr,percent(project_miles_slr,project_miles)))



print("\n#Unconstrained Modal#")
print("Miles where Width > ROW: {0:,} ({1}%)".format(unconst_over, percent(unconst_over,project_miles)))
print("Miles where Width < ROW: {0:,} ({1}%)".format(unconst_under, percent(unconst_under,project_miles)))

print("\n#Constrained Modal#")
print("Miles where Width > ROW: {0:,} ({1}%)".format(const_over, percent(const_over,project_miles)))
print("Miles where Width < ROW: {0:,} ({1}%)".format(const_under, percent(const_under,project_miles) ))


print("\n#Individual Modal Metrics (acres)")
print("\n#Auto#")
print("Exist: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(auto_area_exist,
                                                                           auto_current_value,
                                                                           auto_area_exist_ej,
                                                                           percent(auto_area_exist_ej,auto_area_exist),
                                                                           auto_area_exist_slr,
                                                                           percent(auto_area_exist_slr,auto_area_exist)))
print("Prop: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(auto_area_prop,
                                                                           auto_prop_value,
                                                                           auto_area_prop_ej,
                                                                           percent(auto_area_prop_ej,auto_area_prop),
                                                                           auto_area_prop_slr,
                                                                           percent(auto_area_prop_slr,auto_area_prop)))
print("Const: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(auto_area_const,
                                                                           auto_const_value,
                                                                           auto_area_const_ej,
                                                                           percent(auto_area_const_ej,auto_area_const),
                                                                           auto_area_const_slr,
                                                                           percent(auto_area_const_slr,auto_area_const)))

print("\n#Bike#")
print("Exist: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bike_area_exist,
                                                                           bike_current_value,
                                                                           bike_area_exist_ej,
                                                                           percent(bike_area_exist_ej,bike_area_exist),
                                                                           bike_area_exist_slr,
                                                                           percent(bike_area_exist_slr,bike_area_exist)))

print("Exist (Distinct): {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bike_area_exist_distinct,
                                                                           bike_distinct_current_value,
                                                                           bike_area_exist_distinct_ej,
                                                                           percent(bike_area_exist_distinct_ej,bike_area_exist),
                                                                           bike_area_exist_distinct_slr,
                                                                           percent(bike_area_exist_distinct_slr,bike_area_exist)))

print("Shared Roadway: {0}".format(bike_area_exist_sharedroad))
print("Shared Use Path: {0}".format(bike_area_exist_sharedpath))
print("\nProp: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bike_area_prop,
                                                                           bike_prop_value,
                                                                           bike_area_prop_ej,
                                                                           percent(bike_area_prop_ej,bike_area_prop),
                                                                           bike_area_prop_slr,
                                                                           percent(bike_area_prop_slr,bike_area_prop)))
print("Prop (Distinct): {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bike_area_prop_distinct,
                                                                           bike_prop_distinct_value,
                                                                           bike_area_prop_distinct_ej,
                                                                           percent(bike_area_prop_distinct_ej,bike_area_prop),
                                                                           bike_area_prop_distinct_slr,
                                                                           percent(bike_area_prop_distinct_slr,bike_area_prop)))

print("Shared Roadway: {0}".format(bike_area_prop_sharedroad))
print("Shared Use Path: {0}".format(bike_area_prop_sharedpath))
print("\nConst: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bike_area_const,
                                                                           bike_const_value,
                                                                           bike_area_const_ej,
                                                                           percent(bike_area_const_ej,bike_area_const),
                                                                           bike_area_const_slr,
                                                                           percent(bike_area_const_slr,bike_area_const)))
print("Const (Distinct): {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bike_area_const_distinct,
                                                                           bike_const_distinct_value,
                                                                           bike_area_const_distinct_ej,
                                                                           percent(bike_area_const_distinct_ej,bike_area_const),
                                                                           bike_area_const_distinct_slr,
                                                                           percent(bike_area_const_distinct_slr,bike_area_const)))


print("Shared Roadway: {0}".format(bike_area_const_sharedroad))
print("Shared Use Path: {0}".format(bike_area_const_sharedpath))
print("\n#Medians#")
print("Exist: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(medians_area_exist,
                                                                           medians_current_value,
                                                                           medians_area_exist_ej,
                                                                           percent(medians_area_exist_ej,medians_area_exist),
                                                                           medians_area_exist_slr,
                                                                           percent(medians_area_exist_slr,medians_area_exist)))
print("Prop: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(medians_area_prop,
                                                                           medians_prop_value,
                                                                           medians_area_prop_ej,
                                                                           percent(medians_area_prop_ej,medians_area_prop),
                                                                           medians_area_prop_slr,
                                                                           percent(medians_area_prop_slr,medians_area_prop)))
print("Const: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(medians_area_const,
                                                                           medians_const_value,
                                                                           medians_area_const_ej,
                                                                           percent(medians_area_const_ej,medians_area_const),
                                                                           medians_area_const_slr,
                                                                           percent(medians_area_const_slr,medians_area_const)))


print("\n#Parking#")
print("Exist: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(park_area_exist,
                                                                           park_current_value,
                                                                           park_area_exist_ej,
                                                                           percent(park_area_exist_ej,park_area_exist),
                                                                           park_area_exist_slr,
                                                                           percent(park_area_exist_slr,park_area_exist)))
print("Prop: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(park_area_prop,
                                                                           park_prop_value,
                                                                           park_area_prop_ej,
                                                                           percent(park_area_prop_ej,park_area_prop),
                                                                           park_area_prop_slr,
                                                                           percent(park_area_prop_slr,park_area_prop)))
print("Const: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(park_area_const,
                                                                           park_const_value,
                                                                           park_area_const_ej,
                                                                           percent(park_area_const_ej,park_area_const),
                                                                           park_area_const_slr,
                                                                           percent(park_area_const_slr,park_area_const)))


print("\n#Ped#")
print("Exist: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(ped_area_exist,
                                                                           ped_current_value,
                                                                           ped_area_exist_ej,
                                                                           percent(ped_area_exist_ej,ped_area_exist),
                                                                           ped_area_exist_slr,
                                                                           percent(ped_area_exist_slr,ped_area_exist)))

print("Prop: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(ped_area_prop,
                                                                           ped_prop_value,
                                                                           ped_area_prop_ej,
                                                                           percent(ped_area_prop_ej,ped_area_prop),
                                                                           ped_area_prop_slr,
                                                                           percent(ped_area_prop_slr,ped_area_prop)))
print("Const: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(ped_area_const,
                                                                           ped_const_value,
                                                                           ped_area_const_ej,
                                                                           percent(ped_area_const_ej,ped_area_const),
                                                                           ped_area_const_slr,
                                                                           percent(ped_area_const_slr,ped_area_const)))



print("\n#Bus (dedicated bus lanes - overlaps with Auto area)#")
print("Exist: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bus_area_exist,
                                                                           bus_current_value,
                                                                           bus_area_exist_ej,
                                                                           percent(bus_area_exist_ej,bus_area_exist),
                                                                           bus_area_exist_slr,
                                                                           percent(bus_area_exist_slr,bus_area_exist)))
print("Prop: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bus_area_prop,
                                                                           bus_prop_value,
                                                                           bus_area_prop_ej,
                                                                           percent(bus_area_prop_ej,bus_area_prop),
                                                                           bus_area_prop_slr,
                                                                           percent(bus_area_prop_slr,bus_area_prop)))
print("Const: {0:,} (${1:,}M), EJ: {2:,} ({3}%), SLR: {4:,} ({5}%)".format(bus_area_const,
                                                                           bus_const_value,
                                                                           bus_area_const_ej,
                                                                           percent(bus_area_const_ej,bus_area_const),
                                                                           bus_area_const_slr,
                                                                           percent(bus_area_const_slr,bus_area_const)))






#Totals (all data)#
Miles: 2,059
Acres: 14,838 ($58,870M)
EJ: 539 (26.178%)
SLR: 49 (2.38%)

#Totals (City & Various)#
Miles: 1,449
Acres: 9,603 ($43,584M)
EJ: 303 (20.911%)
SLR: 27 (1.863%)

#Unconstrained Modal#
Miles where Width > ROW: 720 (49.689%)
Miles where Width < ROW: 784 (54.106%)

#Constrained Modal#
Miles where Width > ROW: 225 (15.528%)
Miles where Width < ROW: 1,283 (88.544%)

#Individual Modal Metrics (acres)

#Auto#
Exist: 4,434 ($20,938M), EJ: 889 (20.05%), SLR: 85 (1.917%)
Prop: 4,510 ($21,326M), EJ: 901 (19.978%), SLR: 87 (1.929%)
Const: 4,495 ($21,217M), EJ: 900 (20.022%), SLR: 87 (1.935%)

#Bike#
Exist: 109 ($619M), EJ: 20 (18.349%), SLR: 4 (3.67%)
Prop: 367 ($1,947M), EJ: 80 (21.798%), SLR: 13 (3.542%)
Const: 346 ($1,779M), EJ: 76 (21.965%), SLR: 10 (2.89%)

#Medians#
Exist: 150 ($663M), EJ: 30 (20.0%), SLR: 1 (0.667%)
Prop: 150 ($663M), EJ: 30 (20.0%), SLR: 1 (0.667%)
Const: 150 ($663M), EJ: 30 (20.0%), SLR: 1 (0.667%)

#Parking#
Exist: 1,922 ($8,953M), EJ: 340 (17.69%), SLR: 35 (1.821%)
Prop: 1,922 ($8,953M), EJ: 340 (17.69%), SLR: 35 (1.821%)
Const: 897 ($3,998M), EJ: 207 (23.077%), SLR: 23 (2.564%)

#Ped#
Exist: 1,574 ($8,229M), EJ: 272 (17.281%), SLR: 26 (1.652%)
Prop: 1,988 ($10,151M), EJ: 377 (18.964%), SLR: 35 (1.761%)
Const: 1,941 ($9,765M), EJ: 371 (19.114%), SLR: 32 (1.649%)

#Bus (dedicated bus lanes - overlaps with Auto area)#
Exist: 3 ($39M), EJ: 1 (33.333%), SLR: 0 (0.0%)
Prop: 72 ($696M), EJ: 22 (30.556%), SLR: 7 (9.722%)
Const: 72 ($696M), EJ: 22 (30.556%), SLR: 7 (9.722%)

In [ ]:
round(df["modal_area_auto_exist_LV"].sum()/1000000)

In [ ]:
df

In [ ]:
def convert_acres(series):
    return round(series/43560)

acres(235154181252)

In [ ]:
test = df[["modal_area_bike_exist","modal_area_bike_prop","modal_area_bike_const"]].sum()

convert_acres(test)

In [ ]:
sum_df = df.groupby(["modal_area_bike_exist"])
sum_df.agg("sum").reset_index()

In [ ]:
bike_fields = ["SEGMENTID", "FULLNAME",
               "Fac_Type_BE","modal_width_bike_exist","modal_area_bike_exist",
               "Fac_Type_BP","Fac_Type_BR",
               "modal_width_bike_prop","modal_area_bike_prop",
               "modal_width_bike_const","modal_area_bike_const"]


f_BE = df["Fac_Type_BE"].notnull()
f_BP = df["Fac_Type_BP"].notnull()
f_BE_width = df["modal_width_bike_exist"].notnull()
f_BP_width = df["modal_width_bike_prop"].notnull()
f_BP_area = df["modal_area_bike_prop"].notnull()
f_BR = df["Fac_Type_BR"].notnull()

f_qc = df["modal_width_bike_prop"] < df["modal_width_bike_const"]

df_BE = df.loc[f_BE,bike_fields]
df_BP = df.loc[f_BP,bike_fields]
df_BR = df.loc[f_BR,bike_fields]

df_BE_width = df_BE.groupby(["Fac_Type_BE"]).agg({"modal_width_bike_exist": ["min","mean","max"]})
df_BP_width = df_BP.groupby(["Fac_Type_BP"]).agg({"modal_width_bike_prop": ["min","mean","max"]})
df_BR_width = df_BR.groupby(["Fac_Type_BR"]).agg({"modal_width_bike_prop": ["min","mean","max"]})

df_BE_width.reset_index().plot.bar("Fac_Type_BE", figsize = (12,8))
df_BP_width.reset_index().plot.bar("Fac_Type_BP", figsize = (12,8))
df_BR_width.reset_index().plot.bar("Fac_Type_BR", figsize = (12,8))

In [ ]:
df.loc[f_BP_area,bike_fields]

In [ ]:
df_cols = df.columns.tolist()

df_cols.sort(key=lambda v: (v.upper(), v[0].islower()))
for i in df_cols:
    print(i)

In [ ]:
df["LandValue"].sum()

In [ ]:
#reset all (or some) width and area fields to null (use during development)

fields = [          
    "lane_count_rev",            #0
    "lane_miles_prop",           #1     
    
    "modal_width_auto_const",    #2      
    "modal_area_auto_const",     #3
    
    "modal_width_bike_const",    #4
    "modal_area_bike_const",     #5
    "bike_parallel_route",       #6
    
    "modal_width_medians_const", #7     
    "modal_area_medians_const",  #8
    
    "modal_width_park_const",    #9   
    "modal_area_park_const",     #10
    
    "modal_width_ped_const",     #11 
    "modal_area_ped_const",      #12
    
    "modal_width_bus_const",     #13 
    "modal_area_bus_const",      #14
    
    "modal_width_const",         #15
    "modal_width_const_diff",    #16
    "modal_width_const_desc",    #17
    "modal_area_const",          #18
    "modal_area_const_diff"]     #19

err_count = 0
print("Reseting modal const fields to Null...")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
                                 
            row[0] = None    #
            row[1] = None    # 
            row[2] = None    # 
            row[3] = None    #        
            row[4] = None    # 
            row[5] = None    # 
            row[6] = None    #            
            row[7] = None    # modal_width_bike_exist
            row[8] = None    # modal_width_bike_prop
            row[9] = None    # modal_area_bike_exist
            row[10] = None   # modal_area_bike_prop            
            row[11] = None   # modal_width_medians_exist
            row[12] = None   # modal_width_medians_prop
            row[13] = None   # modal_area_medians_exist
            row[14] = None   # modal_area_medians__prop           
            row[15] = None   # modal_width_park_exist
            row[16] = None   # modal_width_park_prop
            row[17] = None   # modal_area_park_exist
            row[18] = None   # modal_area_park_prop           
            row[19] = None    # modal_width_ped_exist

            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print(error)

print("Done.")
            

In [ ]:
df_fields = ["SEGMENTID", "ej_percent","SLR_percent"]

f_owner = (df["OWNER"] == "CITY") | (df["OWNER"] == "City") | (df["OWNER"] == "VARIOUS")

f_ej = df["ej_percent"] > 0.5

# df.plot.scatter(x="SEGMENTID",y="modal_area_bike_exist",figsize=(20,10), fontsize=12)

df.loc[f_owner & f_ej,df_fields]

In [ ]:
##Compute line percentage metric

print("create layer...")
lyr = arcpy.management.MakeFeatureLayer(SLR, "lyr", where_clause='')

print("dissolve...")
dissolve_output = os.path.join(scratch_gdb_path, "dissolve")
dissolve = arcpy.management.Dissolve(lyr, dissolve_output)

# intersect modal comp w/ selected features
print("intersect w/ modal_comp...")
intersect_output = os.path.join(scratch_gdb_path, "intersect")
intersect = arcpy.analysis.Intersect([modal_composite, dissolve], intersect_output)



#add length field if it doesnt already exist
new_field_name = "SLR_length"
new_field_alias = "Length of Segment inside 3.2ft Sea Level Rise Exposure Area"
print("add {} field...".format(new_field_name))

list_fields = arcpy.ListFields(intersect)
field_names = [i.name for i in list_fields]

if new_field_name in field_names:
    print("{} field already exists".format(new_field_name))
else:
    print("adding field...")
    arcpy.AddField_management(intersect,field_name=new_field_name,field_type="FLOAT",field_alias = new_field_alias)

    
    
#calc field from shape.length with update cursor
print("calc field from shape.length...")
with arcpy.da.UpdateCursor(intersect,["Shape_Length",new_field_name]) as cursor:
    for row in cursor:
        try:
            row[1] = row[0]        
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
            
            
#run frequency on output and sum length fields to normalize data by SegmentID
print("summarize intersect table by SegmentID...")
freq_output = os.path.join(scratch_gdb_path, "freq_output")
freq = arcpy.analysis.Frequency(intersect, freq_output, "SEGMENTID", "SLR_length")



#join intersect freq result back to modal comp
print("Join field to modal_comp on SegmentID...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = freq
join_table_field = "SEGMENTID"
join_fields = ["SLR_length"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)



#add new field for length % if it doesn't already exist
print("add new field for length percentage...")
list_fields = arcpy.ListFields(modal_composite)
field_names = [i.name for i in list_fields]

new_percent_field_name = "SLR_percent"
new_percent_field_alias = "Percent of Segment inside 3.2ft Sea Level Rise Exposure Area"

if "SLR_percent" in field_names:
    print("already exists")
else:
    print("adding percent field...")
    arcpy.AddField_management(modal_composite,field_name=new_percent_field_name,field_type="FLOAT", field_alias = new_percent_field_alias)


#calculate line percentages based on length differences
#calc field from shape.length with update cursor
print("calc percent field...")
with arcpy.da.UpdateCursor(modal_composite,["SLR_length","Shape_Length","SLR_percent","SEGMENTID"]) as cursor:
    for row in cursor:
        try:
            if row[0] is not None:
                row[2] = row[0]/row[1]  
                #print("SegmentID: {0}, Length: {1},  Length: {2}, Ag Percent: {3}".format(row[3], row[1], row[0], row[2]))
                
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
                        
            
print("processing complete")